In [ ]:
!pip install nibabel scikit-learn scipy torchvision torchio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
import torch
from torch.utils.data import Dataset
import torchio as tio

class MRIDataset(Dataset):
    def __init__(self, root_dir, target_shape=(32, 112, 112), ad_repeat=2, augment=True):
        self.file_paths = []
        self.labels = []
        self.target_shape = target_shape
        self.augment = augment

        # Augmentasyon - tüm sınıflara uygulanır
        self.transform = tio.Compose([
            tio.RandomFlip(axes=('LR', 'AP', 'IS'), p=0.5),
            #sol-sağ,ön-arka,alt-üst dönüşüm her birine %50 olasaılık
            tio.RandomAffine(scales=(0.9, 1.1), degrees=10, translation=5, p=0.7),
            #scales=(0.9, 1.1): Görüntüyü %90–110 arasında ölçekler.
            #degrees=10: Her eksende en fazla ±10 derece döndürme.
            #translation=5: Maksimum 5 voxel kaydırma (x, y, z yönünde).
            #p=0.7: Uygulanma olasılığı %70.
            tio.RandomNoise(mean=0.0, std=0.1, p=0.4),
            #Rastgele Gauss gürültüsü ekleyerek modeli gürültüye karşı daha dayanıklı hale getirir.
            tio.RandomBiasField(p=0.3),
            #MRI verilerinde sıkça görülen manyetik alan bozulmalarını taklit eden "bias field" etkisi ekler.
        ])

        class_folders = ['CN_Combined', 'MCI_Combined', 'AD_Combined']
        for label, class_name in enumerate(class_folders):
            class_path = os.path.join(root_dir, class_name)
            if not os.path.exists(class_path): continue

            for subject_folder in os.listdir(class_path):
                full_subject_path = os.path.join(class_path, subject_folder)
                if not os.path.isdir(full_subject_path): continue
                orig_path = os.path.join(full_subject_path, "mri", "orig.mgz")
                if os.path.isfile(orig_path):
                    # AD örneklerini çoğalt (dosya yolu düzeyinde)
                    repeat = ad_repeat if class_name == 'AD_Combined' else 1
                    for _ in range(repeat):
                        self.file_paths.append(orig_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        label = self.labels[idx]

        img = nib.load(path).get_fdata()
        img = (img - np.mean(img)) / np.std(img)

        # Hedef shape'e yeniden boyutlandır
        zoom_factors = [t / s for t, s in zip(self.target_shape, img.shape)]
        img = zoom(img, zoom_factors, order=1)
        img = np.expand_dims(img, axis=0)  # [1, D, H, W]

        # Eğer augmentasyon uygulanacaksa
        if self.augment:
            subject = tio.Subject(mri=tio.ScalarImage(tensor=torch.tensor(img)))
            transformed = self.transform(subject)
            img = transformed.mri.data

        return torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
import torch
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import random

In [ ]:
from torch.utils.data import Dataset, Subset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

# ✅ Dataset sınıfın zaten tanımlı: MRIDataset
data_dir = '/content/drive/MyDrive/Processed'

# Dataset'i yükle (AD çoğaltması dahil)
full_dataset = MRIDataset(data_dir, ad_repeat=2)

# Dataset kontrolü
print(f"Toplam örnek sayısı: {len(full_dataset)}")
if len(full_dataset) > 0:
    print("✅ Dosyalar bulundu. İlk 5 dosya yolu:")
    for i in range(min(5, len(full_dataset))):
        print(full_dataset.file_paths[i])
else:
    raise ValueError("❌ Hiç dosya bulunamadı. Lütfen klasör yapısını ve yolunu kontrol et.")

# ✅ Stratified test set ayırma (%10)
labels = np.array(full_dataset.labels)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
train_val_idx, test_idx = next(sss.split(np.arange(len(full_dataset)), labels))

train_val_dataset = Subset(full_dataset, train_val_idx)
test_dataset = Subset(full_dataset, test_idx)

# Test augmentasyonu kapat
test_dataset.dataset.augment = False

# Test loader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print(f"\n🔍 Dataset Bölümü:")
print(f"Train+Val: {len(train_val_dataset)} | Test: {len(test_dataset)}")


Toplam örnek sayısı: 2635
✅ Dosyalar bulundu. İlk 5 dosya yolu:
/content/drive/MyDrive/Processed/CN_Combined/ADNI_005_S_0602_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20061212103139908_S15966_I32672_CN/mri/orig.mgz
/content/drive/MyDrive/Processed/CN_Combined/ADNI_005_S_0602_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070921133414775_S37063_I74184_CN/mri/orig.mgz
/content/drive/MyDrive/Processed/CN_Combined/ADNI_005_S_0602_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20081022100531771_S53480_I122635_CN/mri/orig.mgz
/content/drive/MyDrive/Processed/CN_Combined/ADNI_005_S_0610_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20061212102204007_S15727_I32667_CN/mri/orig.mgz
/content/drive/MyDrive/Processed/CN_Combined/ADNI_005_S_0610_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20100105121702239_S72045_I162057_CN/mri/orig.mgz

🔍 Dataset Bölümü:
Train+Val: 2371 | Test: 264


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

# 3D DenseLayer: DenseNet'in yapı taşı
class _DenseLayer3D(nn.Module):
    def __init__(self, in_channels, growth_rate, bn_size=4, drop_rate=0):
        super().__init__()
        self.norm1 = nn.BatchNorm3d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv3d(in_channels, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)

        self.norm2 = nn.BatchNorm3d(bn_size * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(bn_size * growth_rate, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)

        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = self.conv1(self.relu1(self.norm1(x)))
        new_features = self.conv2(self.relu2(self.norm2(new_features)))
        if self.drop_rate > 0:
            new_features = F.dropout3d(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


# 3D DenseBlock
class _DenseBlock3D(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate, bn_size=4, drop_rate=0):
        super().__init__()
        layers = []
        for i in range(num_layers):
            layers.append(_DenseLayer3D(in_channels + i * growth_rate, growth_rate, bn_size, drop_rate))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)


# 3D Transition layer
class _Transition3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.norm = nn.BatchNorm3d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.pool = nn.AvgPool3d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(self.relu(self.norm(x)))
        x = self.pool(x)
        return x


# 3D DenseNet modeli
class DenseNet3D(nn.Module):
    def __init__(self, num_classes=3, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv3d(1, num_init_features, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm3d(num_init_features),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        )

        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock3D(num_layers, num_features, growth_rate, bn_size, drop_rate)
            self.features.add_module(f'denseblock{i+1}', block)
            num_features += num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition3D(num_features, num_features // 2)
                self.features.add_module(f'transition{i+1}', trans)
                num_features = num_features // 2

        self.features.add_module('norm_final', nn.BatchNorm3d(num_features))

        self.classifier = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool3d((1, 1, 1)),
            nn.Flatten(),
            nn.Linear(num_features, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes)
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm3d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                init.constant_(m.bias, 0)



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet3D(num_classes=3).to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset, DataLoader
from torch import nn
import torch
import numpy as np
from tqdm import tqdm

# Yeni API
from torch.amp import GradScaler, autocast

NUM_FOLDS = 4
NUM_EPOCHS = 10
BATCH_SIZE = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Stratified K-Fold için etiketleri al
train_val_labels = [full_dataset.labels[i] for i in train_val_idx]
train_val_indices = np.arange(len(train_val_dataset))

skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_val_indices, train_val_labels), 1):
    print(f"\n📁 Fold {fold}/{NUM_FOLDS}")

    train_subset = Subset(train_val_dataset, train_idx)
    val_subset = Subset(train_val_dataset, val_idx)

    # Augmentasyonu sadece eğitim seti için aç
    train_subset.dataset.augment = True
    val_subset.dataset.augment = False

    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    model = DenseNet3D(num_classes=3).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    scaler = GradScaler()  # ✅ Güncel kullanım

    for epoch in range(1, NUM_EPOCHS + 1):
        model.train()
        train_loss, correct, total = 0.0, 0, 0

        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch} [Train]"):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()

            with autocast(device_type='cuda', enabled=DEVICE.type == 'cuda'):  # ✅ Güncel kullanım
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        print(f"Epoch {epoch} - Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")

        # Validation
        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0

        with torch.no_grad():
            for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch} [Val]"):
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                with autocast(device_type='cuda', enabled=DEVICE.type == 'cuda'):
                    outputs = model(imgs)
                    loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = outputs.argmax(dim=1)
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)

        val_acc = correct_val / total_val
        print(f"Epoch {epoch} - Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Fold sonunda test kümesiyle değerlendirme
    model.eval()
    test_loss, correct_test, total_test = 0.0, 0, 0

    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc=f"Fold {fold} [Test]"):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            with autocast(device_type='cuda', enabled=DEVICE.type == 'cuda'):
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            test_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct_test += (preds == labels).sum().item()
            total_test += labels.size(0)

    test_acc = correct_test / total_test
    print(f"✅ Fold {fold} - Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")


📁 Fold 1/4


Epoch 1 [Train]: 100%|██████████| 445/445 [47:57<00:00,  6.47s/it]


Epoch 1 - Train Loss: 496.1296 | Train Acc: 0.4128


Epoch 1 [Val]: 100%|██████████| 149/149 [13:44<00:00,  5.53s/it]


Epoch 1 - Val Loss: 159.5977 | Val Acc: 0.4435


...

Epoch 6 [Val]: 100%|██████████| 148/148 [04:41<00:00,  1.90s/it]


Epoch 6 - Val Loss: 160.9102 | Val Acc: 0.5270


Epoch 7 [Train]: 100%|██████████| 445/445 [14:42<00:00,  1.98s/it]


Epoch 7 - Train Loss: 330.2428 | Train Acc: 0.6757


Epoch 7 [Val]:  56%|█████▌    | 83/148 [02:37<02:05,  1.93s/it]